In [1]:
import tensorflow.keras as keras
import tensorflow as tf
from __future__ import absolute_import
from __future__ import print_function
import numpy as np
import random
from keras.datasets import mnist
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Lambda,Embedding, Reshape, Activation
from tensorflow.keras.layers import Conv2D, Activation,AveragePooling2D,AveragePooling1D,Conv1D, GlobalMaxPooling1D, MaxPooling1D,GlobalAveragePooling1D
from keras import backend as K
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold 

In [2]:
#Siamese Network Implementation 
num_classes = 12
epochs = 20
 
def euclid_dis(vects):
  x,y = vects
  sum_square = K.sum(K.square(x-y), axis=1, keepdims=True)
  return K.sqrt(K.maximum(sum_square, K.epsilon()))
 
def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)
 
def contrastive_loss(y_true, y_pred):
    y_true=tf.cast(y_true, tf.float32)
    y_pred=tf.cast(y_pred, tf.float32)
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)


def create_pairs_new3(x, digit_indices):
    pairs = []
    labels = []
    
    for d in range(num_classes):
        #n'i içeri aldım böylese her class için max loop yapabiliyoruz.
        n=min([len(digit_indices[d])]) -1
        for i in range(n):
            #Random bir Equal index buluyoruz while kullanarak, kendisi(i) haric
            #randomIndiceTrue = random.randrange(0, min([len(digit_indices[d])]))
            randomIndiceTrue=-1
            while True:
                if randomIndiceTrue != i:
                    break
                else:
                    randomIndiceTrue = random.randrange(0, min([len(digit_indices[d])]))
            
            z1, z2 = digit_indices[d][i], digit_indices[d][randomIndiceTrue]
            pairs += [[x[z1], x[z2]]]
            
            inc = random.randrange(1, num_classes)
            dn = (d + inc) % num_classes
            
            #Random bir Non Equal index buluyoruz.
            randomIndiceFalse = random.randrange(0, min([len(digit_indices[dn])]))
            
            z1, z2 = digit_indices[d][i], digit_indices[dn][randomIndiceFalse]
            pairs += [[x[z1], x[z2]]]
            labels += [1,0]
    return np.array(pairs), np.array(labels)



def create_base_net_new2(input_shape):
    height=376
    depth=1
    num_classes=12
    input = Input(shape = input_shape)
    #x = Sequential()(input)
    x = Reshape((height, depth))(input)
    x = Conv1D(12, 3, strides=1, padding='valid', activation='relu')(x)
    x = AveragePooling1D(3)(x)
    x = Conv1D(8, 3, strides=1, padding='valid', activation='relu')(x)
    x = AveragePooling1D(3)(x)
    x = Conv1D(6, 3, strides=1, padding='valid', activation='relu')(x)
    #x = AveragePooling1D(3)(x)
    x = Flatten()(x)
    x = Dense(num_classes, activation = 'relu')(x)
    model = Model(input, x)
    model.summary()
    #print(model.summary())
    return model


def get_f1(y_true, y_pred): 
    pred = K.cast(y_pred < 0.5, y_true.dtype)
    true_positives = K.sum(K.round(K.clip(y_true * pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall)
    return f1_val

def compute_f1(y_true, y_pred):
    pred = y_pred.ravel() < 0.5
    pred=pred*1
    TN,FP,FN,TP=confusion_matrix(y_true, pred).ravel()

    recall = TP/(TP+FN)
    precision=TP/(TP+FP)

    f1_val = 2*(precision*recall)/(precision+recall)
    return f1_val

def compute_cosine_f1(t_pairs, t_y):
    cosine_threshold=0.6
    cTP=0
    cFP=0
    cFN=0
    cTN=0
    for i in range(np.shape(t_y)[0]):
        cos_result=cosine_similarity(t_pairs[i, 0].reshape(1, -1), t_pairs[i, 1].reshape(1, -1))
        if t_y[i]==1 and cos_result>=cosine_threshold:
            cTP=cTP+1
        elif t_y[i]!=1 and cos_result>=cosine_threshold:
            cFP=cFP+1
        elif t_y[i]==1 and cos_result<cosine_threshold:
            cFN=cFN+1
        elif t_y[i]!=1 and cos_result<cosine_threshold:
            cTN=cTN+1
        
    recall = cTP/(cTP+cFN)
    
    precision=cTP/(cTP+cFP)
    f1_cosine_val = 2*(precision*recall)/(precision+recall)
    return f1_cosine_val
    

In [3]:
#Dataset
df=pd.read_csv('db_dep_dataset_1_noname.csv',delimiter=';',header=None)
df
# Table names are anonymized.
# First column: TABLE_UniqueTableNumber_ModuleNumber
# Second column: Class number. Equal to Module Number-1. There are 12 classes in totally.
# Third column: Unique table number

,0,1,2,3,4,5,6,7,8,9,...,369,370,371,372,373,374,375,376,377,378
0,TABLE_1_1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,TABLE_2_1,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,TABLE_3_1,0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,TABLE_4_1,0,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,TABLE_5_1,0,5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,TABLE_116_12,11,116,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
116,TABLE_117_12,11,117,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
117,TABLE_118_12,11,118,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
118,TABLE_119_12,11,119,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Separation X and Y values
Y = df[1]

Y=Y.values

#First three columns removed.
X = df
X = X.drop(0,axis=1)
X = X.drop(1,axis=1)
X = X.drop(2,axis=1)

X=X.values

In [5]:
# Siamese Neural Network Training & Validation Step (Process Group 1)


kfold = StratifiedKFold(n_splits=5, shuffle=True)
cvTrainscores = []
cvTestscores = []
cvi=0

cvTrainscoresCosine = []
cvTestscoresCosine = []


for train, test in kfold.split(X, Y):

    cvi=cvi+1
    print("n-fold: "+str(cvi))
    
    x_train = X[train]
    x_test = X[test]
    y_train = Y[train]
    y_test = Y[test]
    
    x_train = x_train.reshape(x_train.shape[0], 376, 1)
    x_test = x_test.reshape(x_test.shape[0], 376, 1)
    input_shape = (376, 1)
    print(x_train.shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    y_train = y_train.astype('float32')
    y_test = y_test.astype('float32')
    input_shape = (376,1)
    
    print(input_shape)
    
    # create training+test positive and negative pairs
    digit_indices = [np.where(y_train == i)[0] for i in range(num_classes)]
    tr_pairs, tr_y = create_pairs_new3(x_train, digit_indices)
    
    digit_indices = [np.where(y_test == i)[0] for i in range(num_classes)]
    te_pairs, te_y = create_pairs_new3(x_test, digit_indices)
    
    # network definition
    base_network = create_base_net_new2(input_shape)
    
    print(input_shape)

    input_a = Input(shape=input_shape)
    input_b = Input(shape=input_shape)
    
    processed_a = base_network(input_a)
    processed_b = base_network(input_b)
    
    distance = Lambda(euclid_dis,
                  output_shape=eucl_dist_output_shape)([processed_a, processed_b])
    
    model = Model([input_a, input_b], distance)

    #train
    model.compile(loss=contrastive_loss, optimizer='adam', metrics=[get_f1])

    model.fit([tr_pairs[:, 0], tr_pairs[:, 1]], tr_y,
          batch_size=16,
          epochs=epochs,
          validation_data=([te_pairs[:, 0], te_pairs[:, 1]], te_y))
    
    # compute final accuracy on training and test sets for n-fold

    y_pred = model.predict([tr_pairs[:, 0], tr_pairs[:, 1]])
    tr_acc = compute_f1(tr_y, y_pred)
    tr_f1_cosine = compute_cosine_f1(tr_pairs,tr_y)

    y_pred = model.predict([te_pairs[:, 0], te_pairs[:, 1]])
    te_acc = compute_f1(te_y, y_pred)
    te_f1_cosine = compute_cosine_f1(te_pairs,te_y)
    
  
    cvTrainscores.append(100 * tr_acc)
    cvTestscores.append(100 * te_acc)

    cvTrainscoresCosine.append(100 * tr_f1_cosine)
    cvTestscoresCosine.append(100 * te_f1_cosine)
    
    
    



n-fold: 1
(96, 376, 1)
(376, 1)
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 376, 1)]          0         
_________________________________________________________________
reshape (Reshape)            (None, 376, 1)            0         
_________________________________________________________________
conv1d (Conv1D)              (None, 374, 12)           48        
_________________________________________________________________
average_pooling1d (AveragePo (None, 124, 12)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 122, 8)            296       
_________________________________________________________________
average_pooling1d_1 (Average (None, 40, 8)             0         
_________________________________________________________________
conv1d_2 (Conv1D)     

Epoch 18/20
11/11 [==============================] - 0s 5ms/step - loss: 0.0198 - get_f1: 1.0000 - val_loss: 0.0391 - val_get_f1: 0.9706
Epoch 19/20
11/11 [==============================] - 0s 5ms/step - loss: 0.0187 - get_f1: 1.0000 - val_loss: 0.0386 - val_get_f1: 0.9706
Epoch 20/20
11/11 [==============================] - 0s 4ms/step - loss: 0.0179 - get_f1: 1.0000 - val_loss: 0.0385 - val_get_f1: 0.9706
n-fold: 3
(96, 376, 1)
(376, 1)
Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 376, 1)]          0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 376, 1)            0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 374, 12)           48        
_________________________________________________________________
average

Epoch 15/20
11/11 [==============================] - 0s 4ms/step - loss: 0.0201 - get_f1: 0.9846 - val_loss: 0.0307 - val_get_f1: 1.0000
Epoch 16/20
11/11 [==============================] - 0s 5ms/step - loss: 0.0184 - get_f1: 0.9848 - val_loss: 0.0304 - val_get_f1: 1.0000
Epoch 17/20
11/11 [==============================] - 0s 4ms/step - loss: 0.0171 - get_f1: 0.9816 - val_loss: 0.0295 - val_get_f1: 1.0000
Epoch 18/20
11/11 [==============================] - 0s 4ms/step - loss: 0.0160 - get_f1: 0.9870 - val_loss: 0.0290 - val_get_f1: 1.0000
Epoch 19/20
11/11 [==============================] - 0s 4ms/step - loss: 0.0150 - get_f1: 0.9893 - val_loss: 0.0288 - val_get_f1: 1.0000
Epoch 20/20
11/11 [==============================] - 0s 4ms/step - loss: 0.0141 - get_f1: 0.9869 - val_loss: 0.0285 - val_get_f1: 1.0000
n-fold: 5
(96, 376, 1)
(376, 1)
Model: "functional_17"
_________________________________________________________________
Layer (type)                 Output Shape              Pa

In [6]:
# Print final F1 Score accuracy on training and test sets based on Cross Validation
print("Siamese Cross Validation F1 Result:")
print("Train: %.2f%% (+/- %.2f%%)" % (np.mean(cvTrainscores), np.std(cvTrainscores)))
print("Test: %.2f%% (+/- %.2f%%)" % (np.mean(cvTestscores), np.std(cvTestscores)))

print("Cosine Cross Validation F1 Result:")
print("Train: %.2f%% (+/- %.2f%%)" % (np.mean(cvTrainscoresCosine), np.std(cvTrainscoresCosine)))
print("Test: %.2f%% (+/- %.2f%%)" % (np.mean(cvTestscoresCosine), np.std(cvTestscoresCosine)))

Siamese Cross Validation F1 Result:
Train: 99.41% (+/- 0.37%)
Test: 95.86% (+/- 2.64%)
Cosine Cross Validation F1 Result:
Train: 92.29% (+/- 1.23%)
Test: 94.46% (+/- 5.51%)


In [7]:
# Similarity Calculation Step (Process Group 2)
import csv
table_name = df[0]
with open('pairwise_similarity_calculation_result.txt','w') as f1:
    writer=csv.writer(f1, delimiter=';',lineterminator='\n',)
    for i in range(np.shape(X)[0]):
        for y in range(np.shape(X)[0]):
            siamese_result = model.predict([X[i].reshape(1,376,1), X[y].reshape(1,376,1)])
            print(table_name[i]+';'+table_name[y]+';'+' '.join(map(str, siamese_result[0])))
            writer.writerow(list([table_name[i]+';'+table_name[y]+';'+' '.join(map(str, siamese_result[0]))]))

TABLE_1_1;TABLE_1_1;0.00031622776
TABLE_1_1;TABLE_2_1;0.1481434
TABLE_1_1;TABLE_3_1;0.1481434
TABLE_1_1;TABLE_4_1;0.30352515
TABLE_1_1;TABLE_5_1;0.30352515
TABLE_1_1;TABLE_6_1;0.22947446
TABLE_1_1;TABLE_7_1;0.33151835
TABLE_1_1;TABLE_8_1;0.3363494
TABLE_1_1;TABLE_9_1;0.33151835
TABLE_1_1;TABLE_10_1;0.33151835
TABLE_1_1;TABLE_11_2;0.7587728
TABLE_1_1;TABLE_12_2;0.72187597
TABLE_1_1;TABLE_13_2;0.72187597
TABLE_1_1;TABLE_14_2;0.8547809
TABLE_1_1;TABLE_15_2;0.80899274
TABLE_1_1;TABLE_16_2;0.7913098
TABLE_1_1;TABLE_17_2;0.90204275
TABLE_1_1;TABLE_18_2;0.8564565
TABLE_1_1;TABLE_19_2;0.74166536
TABLE_1_1;TABLE_20_2;0.67818564
TABLE_1_1;TABLE_21_3;1.053404
TABLE_1_1;TABLE_22_3;0.9731426
TABLE_1_1;TABLE_23_3;1.0076264
TABLE_1_1;TABLE_24_3;1.0076264
TABLE_1_1;TABLE_25_3;1.0076264
TABLE_1_1;TABLE_26_3;1.0527599
TABLE_1_1;TABLE_27_3;0.9733198
TABLE_1_1;TABLE_28_3;0.9203377
TABLE_1_1;TABLE_29_3;1.0680825
TABLE_1_1;TABLE_30_3;1.0680825
TABLE_1_1;TABLE_31_4;0.8926955
TABLE_1_1;TABLE_32_4;0.9241388
TA

TABLE_3_1;TABLE_25_3;1.081022
TABLE_3_1;TABLE_26_3;1.1181407
TABLE_3_1;TABLE_27_3;1.0491759
TABLE_3_1;TABLE_28_3;1.002723
TABLE_3_1;TABLE_29_3;1.1364267
TABLE_3_1;TABLE_30_3;1.1364267
TABLE_3_1;TABLE_31_4;0.9492758
TABLE_3_1;TABLE_32_4;0.9853342
TABLE_3_1;TABLE_33_4;0.9570155
TABLE_3_1;TABLE_34_4;0.8998817
TABLE_3_1;TABLE_35_4;0.9639196
TABLE_3_1;TABLE_36_4;0.9611824
TABLE_3_1;TABLE_37_4;0.93996394
TABLE_3_1;TABLE_38_4;0.9272884
TABLE_3_1;TABLE_39_4;1.0153975
TABLE_3_1;TABLE_40_4;0.9381483
TABLE_3_1;TABLE_41_5;0.86372393
TABLE_3_1;TABLE_42_5;0.8123453
TABLE_3_1;TABLE_43_5;0.8123453
TABLE_3_1;TABLE_44_5;0.8123453
TABLE_3_1;TABLE_45_5;0.8123453
TABLE_3_1;TABLE_46_5;0.8261123
TABLE_3_1;TABLE_47_5;0.8261123
TABLE_3_1;TABLE_48_5;0.8261123
TABLE_3_1;TABLE_49_5;0.852418
TABLE_3_1;TABLE_50_5;0.8211793
TABLE_3_1;TABLE_51_6;0.46055737
TABLE_3_1;TABLE_52_6;0.5863898
TABLE_3_1;TABLE_53_6;0.46228033
TABLE_3_1;TABLE_54_6;0.49018982
TABLE_3_1;TABLE_55_6;0.5581272
TABLE_3_1;TABLE_56_6;0.48570284
TABLE

TABLE_5_1;TABLE_46_5;0.9428704
TABLE_5_1;TABLE_47_5;0.9428704
TABLE_5_1;TABLE_48_5;0.9428704
TABLE_5_1;TABLE_49_5;0.97313166
TABLE_5_1;TABLE_50_5;0.94026774
TABLE_5_1;TABLE_51_6;0.55746
TABLE_5_1;TABLE_52_6;0.6874294
TABLE_5_1;TABLE_53_6;0.57472503
TABLE_5_1;TABLE_54_6;0.5921667
TABLE_5_1;TABLE_55_6;0.66719884
TABLE_5_1;TABLE_56_6;0.59220946
TABLE_5_1;TABLE_57_6;0.6074543
TABLE_5_1;TABLE_58_6;0.75958425
TABLE_5_1;TABLE_59_6;0.6237858
TABLE_5_1;TABLE_60_6;0.6134815
TABLE_5_1;TABLE_61_7;0.697235
TABLE_5_1;TABLE_62_7;0.6625179
TABLE_5_1;TABLE_63_7;0.71641535
TABLE_5_1;TABLE_64_7;0.65117383
TABLE_5_1;TABLE_65_7;0.71997553
TABLE_5_1;TABLE_66_7;0.7078396
TABLE_5_1;TABLE_67_7;0.7078396
TABLE_5_1;TABLE_68_7;0.71539605
TABLE_5_1;TABLE_69_7;0.71539605
TABLE_5_1;TABLE_70_7;0.71539605
TABLE_5_1;TABLE_71_8;1.1074573
TABLE_5_1;TABLE_72_8;1.1587652
TABLE_5_1;TABLE_73_8;1.1587652
TABLE_5_1;TABLE_74_8;1.1587652
TABLE_5_1;TABLE_75_8;1.1587652
TABLE_5_1;TABLE_76_8;1.1734537
TABLE_5_1;TABLE_77_8;1.1734537

TABLE_7_1;TABLE_68_7;0.73041165
TABLE_7_1;TABLE_69_7;0.73041165
TABLE_7_1;TABLE_70_7;0.73041165
TABLE_7_1;TABLE_71_8;1.157188
TABLE_7_1;TABLE_72_8;1.2067982
TABLE_7_1;TABLE_73_8;1.2067982
TABLE_7_1;TABLE_74_8;1.2067982
TABLE_7_1;TABLE_75_8;1.2067982
TABLE_7_1;TABLE_76_8;1.2209857
TABLE_7_1;TABLE_77_8;1.2209857
TABLE_7_1;TABLE_78_8;1.278515
TABLE_7_1;TABLE_79_8;1.2352958
TABLE_7_1;TABLE_80_8;1.2352958
TABLE_7_1;TABLE_81_9;0.9714657
TABLE_7_1;TABLE_82_9;1.0360543
TABLE_7_1;TABLE_83_9;0.92611104
TABLE_7_1;TABLE_84_9;0.9011513
TABLE_7_1;TABLE_85_9;1.0002619
TABLE_7_1;TABLE_86_9;0.93708026
TABLE_7_1;TABLE_87_9;0.941345
TABLE_7_1;TABLE_88_9;0.99013865
TABLE_7_1;TABLE_89_9;0.91793597
TABLE_7_1;TABLE_90_9;0.99430645
TABLE_7_1;TABLE_91_10;1.1771677
TABLE_7_1;TABLE_92_10;1.327352
TABLE_7_1;TABLE_93_10;1.3118882
TABLE_7_1;TABLE_94_10;1.3239015
TABLE_7_1;TABLE_95_10;1.2324382
TABLE_7_1;TABLE_96_10;1.3505063
TABLE_7_1;TABLE_97_10;1.2937535
TABLE_7_1;TABLE_98_10;1.5003774
TABLE_7_1;TABLE_99_10;1.567

TABLE_9_1;TABLE_88_9;0.99013865
TABLE_9_1;TABLE_89_9;0.91793597
TABLE_9_1;TABLE_90_9;0.99430645
TABLE_9_1;TABLE_91_10;1.1771677
TABLE_9_1;TABLE_92_10;1.327352
TABLE_9_1;TABLE_93_10;1.3118882
TABLE_9_1;TABLE_94_10;1.3239015
TABLE_9_1;TABLE_95_10;1.2324382
TABLE_9_1;TABLE_96_10;1.3505063
TABLE_9_1;TABLE_97_10;1.2937535
TABLE_9_1;TABLE_98_10;1.5003774
TABLE_9_1;TABLE_99_10;1.5673189
TABLE_9_1;TABLE_100_10;1.3283403
TABLE_9_1;TABLE_101_11;1.2691227
TABLE_9_1;TABLE_102_11;1.1740927
TABLE_9_1;TABLE_103_11;1.2464395
TABLE_9_1;TABLE_104_11;1.2464395
TABLE_9_1;TABLE_105_11;1.2730674
TABLE_9_1;TABLE_106_11;1.2730674
TABLE_9_1;TABLE_107_11;1.2730674
TABLE_9_1;TABLE_108_11;1.2798054
TABLE_9_1;TABLE_109_11;1.246238
TABLE_9_1;TABLE_110_11;1.3685391
TABLE_9_1;TABLE_111_12;0.92546266
TABLE_9_1;TABLE_112_12;0.93968254
TABLE_9_1;TABLE_113_12;0.95511234
TABLE_9_1;TABLE_114_12;0.83971864
TABLE_9_1;TABLE_115_12;1.0102801
TABLE_9_1;TABLE_116_12;0.8553664
TABLE_9_1;TABLE_117_12;0.85695624
TABLE_9_1;TABLE_118

...

TABLE_111_12;TABLE_3_1;0.8037119
TABLE_111_12;TABLE_4_1;0.89327157
TABLE_111_12;TABLE_5_1;0.89327157
TABLE_111_12;TABLE_6_1;0.78356445
TABLE_111_12;TABLE_7_1;0.92546266
TABLE_111_12;TABLE_8_1;0.97469354
TABLE_111_12;TABLE_9_1;0.92546266
TABLE_111_12;TABLE_10_1;0.92546266
TABLE_111_12;TABLE_11_2;1.248877
TABLE_111_12;TABLE_12_2;1.2274412
TABLE_111_12;TABLE_13_2;1.2274412
TABLE_111_12;TABLE_14_2;1.322832
TABLE_111_12;TABLE_15_2;1.2953697
TABLE_111_12;TABLE_16_2;1.2578142
TABLE_111_12;TABLE_17_2;1.3606702
TABLE_111_12;TABLE_18_2;1.33024
TABLE_111_12;TABLE_19_2;1.2542318
TABLE_111_12;TABLE_20_2;1.1780497
TABLE_111_12;TABLE_21_3;1.2781938
TABLE_111_12;TABLE_22_3;1.2683717
TABLE_111_12;TABLE_23_3;1.3123751
TABLE_111_12;TABLE_24_3;1.3123751
TABLE_111_12;TABLE_25_3;1.3123751
TABLE_111_12;TABLE_26_3;1.3808007
TABLE_111_12;TABLE_27_3;1.252256
TABLE_111_12;TABLE_28_3;1.1685809
TABLE_111_12;TABLE_29_3;1.3695033
TABLE_111_12;TABLE_30_3;1.3695033
TABLE_111_12;TABLE_31_4;0.65764177
TABLE_111_12;TABLE

TABLE_113_12;TABLE_4_1;0.92965305
TABLE_113_12;TABLE_5_1;0.92965305
TABLE_113_12;TABLE_6_1;0.8224422
TABLE_113_12;TABLE_7_1;0.95511234
TABLE_113_12;TABLE_8_1;1.0150418
TABLE_113_12;TABLE_9_1;0.95511234
TABLE_113_12;TABLE_10_1;0.95511234
TABLE_113_12;TABLE_11_2;1.2949511
TABLE_113_12;TABLE_12_2;1.2862029
TABLE_113_12;TABLE_13_2;1.2862029
TABLE_113_12;TABLE_14_2;1.377077
TABLE_113_12;TABLE_15_2;1.3402549
TABLE_113_12;TABLE_16_2;1.3045264
TABLE_113_12;TABLE_17_2;1.418129
TABLE_113_12;TABLE_18_2;1.3807026
TABLE_113_12;TABLE_19_2;1.2997336
TABLE_113_12;TABLE_20_2;1.2246182
TABLE_113_12;TABLE_21_3;1.2224302
TABLE_113_12;TABLE_22_3;1.2456795
TABLE_113_12;TABLE_23_3;1.2738789
TABLE_113_12;TABLE_24_3;1.2738789
TABLE_113_12;TABLE_25_3;1.2738789
TABLE_113_12;TABLE_26_3;1.3550571
TABLE_113_12;TABLE_27_3;1.2233454
TABLE_113_12;TABLE_28_3;1.1351799
TABLE_113_12;TABLE_29_3;1.3364314
TABLE_113_12;TABLE_30_3;1.3364314
TABLE_113_12;TABLE_31_4;0.54663247
TABLE_113_12;TABLE_32_4;0.54775465
TABLE_113_12;TA

TABLE_115_12;TABLE_5_1;0.986057
TABLE_115_12;TABLE_6_1;0.888516
TABLE_115_12;TABLE_7_1;1.0102801
TABLE_115_12;TABLE_8_1;1.0780265
TABLE_115_12;TABLE_9_1;1.0102801
TABLE_115_12;TABLE_10_1;1.0102801
TABLE_115_12;TABLE_11_2;1.3596207
TABLE_115_12;TABLE_12_2;1.3496635
TABLE_115_12;TABLE_13_2;1.3496635
TABLE_115_12;TABLE_14_2;1.4396385
TABLE_115_12;TABLE_15_2;1.4040215
TABLE_115_12;TABLE_16_2;1.369438
TABLE_115_12;TABLE_17_2;1.4804677
TABLE_115_12;TABLE_18_2;1.4426634
TABLE_115_12;TABLE_19_2;1.3630419
TABLE_115_12;TABLE_20_2;1.2899122
TABLE_115_12;TABLE_21_3;1.2609918
TABLE_115_12;TABLE_22_3;1.290459
TABLE_115_12;TABLE_23_3;1.3200732
TABLE_115_12;TABLE_24_3;1.3200732
TABLE_115_12;TABLE_25_3;1.3200732
TABLE_115_12;TABLE_26_3;1.4026036
TABLE_115_12;TABLE_27_3;1.2678999
TABLE_115_12;TABLE_28_3;1.1761403
TABLE_115_12;TABLE_29_3;1.3833938
TABLE_115_12;TABLE_30_3;1.3833938
TABLE_115_12;TABLE_31_4;0.579906
TABLE_115_12;TABLE_32_4;0.5751994
TABLE_115_12;TABLE_33_4;0.5944015
TABLE_115_12;TABLE_34_4;

TABLE_117_12;TABLE_7_1;0.85695624
TABLE_117_12;TABLE_8_1;0.8880675
TABLE_117_12;TABLE_9_1;0.85695624
TABLE_117_12;TABLE_10_1;0.85695624
TABLE_117_12;TABLE_11_2;1.1543822
TABLE_117_12;TABLE_12_2;1.1279196
TABLE_117_12;TABLE_13_2;1.1279196
TABLE_117_12;TABLE_14_2;1.2260312
TABLE_117_12;TABLE_15_2;1.2020473
TABLE_117_12;TABLE_16_2;1.1635648
TABLE_117_12;TABLE_17_2;1.2635216
TABLE_117_12;TABLE_18_2;1.2340866
TABLE_117_12;TABLE_19_2;1.160236
TABLE_117_12;TABLE_20_2;1.0838885
TABLE_117_12;TABLE_21_3;1.2688618
TABLE_117_12;TABLE_22_3;1.2341478
TABLE_117_12;TABLE_23_3;1.2870537
TABLE_117_12;TABLE_24_3;1.2870537
TABLE_117_12;TABLE_25_3;1.2870537
TABLE_117_12;TABLE_26_3;1.3451632
TABLE_117_12;TABLE_27_3;1.2240667
TABLE_117_12;TABLE_28_3;1.1463373
TABLE_117_12;TABLE_29_3;1.3405323
TABLE_117_12;TABLE_30_3;1.3405323
TABLE_117_12;TABLE_31_4;0.7359787
TABLE_117_12;TABLE_32_4;0.7514635
TABLE_117_12;TABLE_33_4;0.7536831
TABLE_117_12;TABLE_34_4;0.7200081
TABLE_117_12;TABLE_35_4;0.73168075
TABLE_117_12;T

TABLE_119_12;TABLE_9_1;0.9844139
TABLE_119_12;TABLE_10_1;0.9844139
TABLE_119_12;TABLE_11_2;1.3052363
TABLE_119_12;TABLE_12_2;1.2868829
TABLE_119_12;TABLE_13_2;1.2868829
TABLE_119_12;TABLE_14_2;1.3823394
TABLE_119_12;TABLE_15_2;1.3431454
TABLE_119_12;TABLE_16_2;1.315472
TABLE_119_12;TABLE_17_2;1.4258544
TABLE_119_12;TABLE_18_2;1.3825161
TABLE_119_12;TABLE_19_2;1.3083264
TABLE_119_12;TABLE_20_2;1.2330545
TABLE_119_12;TABLE_21_3;1.2268306
TABLE_119_12;TABLE_22_3;1.2296716
TABLE_119_12;TABLE_23_3;1.2731076
TABLE_119_12;TABLE_24_3;1.2731076
TABLE_119_12;TABLE_25_3;1.2731076
TABLE_119_12;TABLE_26_3;1.3459699
TABLE_119_12;TABLE_27_3;1.2098063
TABLE_119_12;TABLE_28_3;1.1234691
TABLE_119_12;TABLE_29_3;1.3309782
TABLE_119_12;TABLE_30_3;1.3309782
TABLE_119_12;TABLE_31_4;0.6577909
TABLE_119_12;TABLE_32_4;0.6615275
TABLE_119_12;TABLE_33_4;0.67218363
TABLE_119_12;TABLE_34_4;0.6559274
TABLE_119_12;TABLE_35_4;0.64977676
TABLE_119_12;TABLE_36_4;0.6929288
TABLE_119_12;TABLE_37_4;0.5153936
TABLE_119_12;T